第九章 迭代器，生成器与装饰器
--------------------------------------------
to do list
- [ ] 自定义一个迭代器，利用for循环进行索引
- [ ] itertools常用的函数
- [ ] 生成器
- [ ] 生产者和消费者模型
- [ ] 装饰器修饰函数
- [ ] 装饰器修饰类

### 迭代器
- python中迭代器的使用是最为广泛的，凡是使用for语句，其本质上都是迭代器的应用
- 从代码的角度看，迭代器是实现了迭代器协议方法的对象或类。迭代器协议方法主要是两个
    - \_\_iter\_\_(), 方法返回对象本身，它是for语句使用迭代器的要求
    - \_\_next\_\_(), 方法用于返回容器中下一个元素或数据。当容器中的数据用尽时，应该引发StopIteration异常。

- 自定义迭代器

In [1]:
class MyIterator:
    def __init__(self, x=2, xmax=100):
        self.__mul, self.__x = x, x
        self.__xmax = xmax

    def __iter__(self):
        return self

    def __next__(self):
        if self.__x and self.__x != 1:
            self.__mul *= self.__x
            if self.__mul <= self.__xmax:
                return self.__mul
            else:
                raise StopIteration
        else:
            raise StopIteration


myIter = MyIterator()
for i in myIter:
    print('迭代的数据元素为:', i)

迭代的数据元素为: 4
迭代的数据元素为: 8
迭代的数据元素为: 16
迭代的数据元素为: 32
迭代的数据元素为: 64


- 内建迭代器函数 iter(),另外在标准库的itertools模块中还有丰富的迭代器工具，他们存在于itertools中。
- itertools模块中提供了近二十个迭代器工具函数，主要有三类
    - 无限迭代器
    - 迭代短序列
    - 组合迭代序列

In [10]:
import itertools
for i in itertools.count(1, 3):  # 从1开始，以3位步进行计数迭代
    print(i)
    if i >= 10:
        break

1
4
7
10


In [11]:
x = 0
for i in itertools.cycle(['a', 'b']):  #无限循环迭代['a','b']
    print(i)
    x += 1
    if x > 6:
        break

a
b
a
b
a
b
a


In [12]:
list(itertools.repeat(3, 3))

[3, 3, 3]

In [13]:
list(itertools.chain([1, 2], [2, 3]))  # 将两个列表连接起来进行迭代

[1, 2, 2, 3]

In [14]:
print(list(itertools.compress([1, 2, 3, 4], [1, [], True, 3])))
#根据后边selectin中的真假来决定是否输出前边data的数值
print(list(itertools.compress([1, 2, 3, 4], [1, [], True, 1])))

[1, 3, 4]
[1, 3, 4]


In [15]:
list(itertools.dropwhile(lambda x: x > 6, [8, 9, 1, 2, 8, 9]))
#第一个参数的处理结果为假时开始迭代后边的数值

[1, 2, 8, 9]

In [16]:
list(itertools.takewhile(lambda x: x > 10, [18, 19, 1, 21, 8, 9]))

[18, 19]

In [17]:
for its in itertools.tee([0, 1], 2):  #将[0,1]进行2次迭代
    for it in its:
        print(it)

0
1
0
1


In [18]:
list(itertools.permutations('abc', 2))

[('a', 'b'), ('a', 'c'), ('b', 'a'), ('b', 'c'), ('c', 'a'), ('c', 'b')]

In [19]:
list(itertools.combinations('abc', 2))

[('a', 'b'), ('a', 'c'), ('b', 'c')]

### 生成器
- 生成器可以生成一个值的序列用于迭代，并且这个值的序列不是一次生成的，而是使用一个，生成一个值的序列
>一个带有 yield 的函数就是一个 generator，它和普通函数不同，生成一个 generator 看起来像函数调用，但不会执行任何函数代码，直到对其调用 next()（在 for 循环中会自动调用 next()）才开始执行。虽然执行流程仍按函数的流程执行，但每执行到一个 yield 语句就会中断，并返回一个迭代值，下次执行时从 yield 的下一个语句继续执行。看起来就好像一个函数在正常执行的过程中被 yield 中断了数次，每次中断都会通过 yield 返回当前的迭代值。

- 生成器的实例化时并不会立即执行，而是等待调用其\_\_next\_\_()方法才开始运行

In [20]:
def myYield(n):  #定义生成器函数
    while n > 0:
        print("开始生成...")
        yield n  #yield语句，产生返回值， 程序执行到这里，会暂停，下次调用继续执行
        print("完成一次...")
        n = n - 1


for i in myYield(4):  #利用for循环对生成器进行索引
    print(i)
my_yield = myYield(3)
print("已经实例化了对象")
my_yield.__next__()
print(my_yield.__next__())

开始生成...
4
完成一次...
开始生成...
3
完成一次...
开始生成...
2
完成一次...
开始生成...
1
完成一次...
已经实例化了对象
开始生成...
完成一次...
开始生成...
2


- 可以接收调用者传来的值并重新初始化生成器生成的值

In [21]:
def myYield(n):
    while n > 0:
        rcv = yield (n)  #yield 语句， 用于函数返回，同时也可以也接受send函数发过来的数据
        n = n - 1
        if rcv is not None:
            n = rcv


my_yield = myYield(3)
print(my_yield.__next__())
print(my_yield.__next__())
print("传递给my_yield一个10，重新初始化生成器。")
print(my_yield.send(10))
print(my_yield.__next__())

3
2
传递给my_yield一个10，重新初始化生成器。
10
9


In [22]:
a = (i**2 for i in range(5))  # 其本质也是一个生成器
print(a.__next__())
print(a.__next__())
print(a.__next__())
a.send(6) # 此函数不起作用
print('type(a)', type(a))
print('list(a)', list(a)) # 利用list可以将生成器转换为list

a = (i**2 for i in range(5))  # 其本质也是一个生成器，可用for循环进行迭代
for i in a:
    print(i)

0
1
4
type(a) <class 'generator'>
list(a) [16]
0
1
4
9
16


### 装饰器
- 装饰器是一种增加函数或类的功能的简单方法，它可以快速地给不同的函数或类插入相同的功能。从本质上说，它是一种代码实现方式

In [23]:
def abc(fun):  #装饰器的参数是函数
    def wrapper(*args, **kwargs): #定义装饰类函数
        print("开始运行")
        fun(*args, **kwargs)
        print("结束运行！")

    return wrapper #返回一个装饰器


@abc
def demo_decoration(x):
    a = []
    for i in range(x):
        a.append(i)
    print(a)


@abc
def hello(name):
    print("hello", name)


demo_decoration(3)
print()
hello("john")

开始运行
[0, 1, 2]
结束运行！

开始运行
hello john
结束运行！


In [2]:
#装饰器也可以携带参数
def abc(action):  #装饰器携带参数
    def mabc(fun):
        def wrapper(*args, **kwargs):  #定义装饰类函数
            print("开始运行",action)
            fun(*args, **kwargs)
            print("结束运行！",action)

        return wrapper  #返回一个装饰器

    return mabc


@abc('demo')
def demo_decoration(x):
    a = []
    for i in range(x):
        a.append(i)
    print(a)


@abc('hello')
def hello(name):
    print("hello", name)


demo_decoration(3)
print()
hello("john")

开始运行 demo
[0, 1, 2]
结束运行！ demo

开始运行 hello
hello john
结束运行！ hello


- 装饰器修饰类

In [3]:
def abc(myclass):  # 在函数内定义内部类
    class InnerClass:
        def __init__(self, z=0):
            self.z = 0
            self.wrapper = myclass()  # 实例化传入的类

        def position(self):
            self.wrapper.position()  # 先调用传入类的position方法
            print('z axis:', self.z)

    return InnerClass


@abc
class coordination:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def position(self):
        print('x axis:', self.x)
        print('y axis:', self.y)


coor = coordination()
coor.position()

x axis: 0
y axis: 0
z axis: 0


### 生成器与协程
- 生产者和消费者模型（yield用于接收数据，send用于发送数据）

In [26]:
def consumer(): #有yield的函数即为生成器
    print('等待接受处理任务...')
    while True:
        data = (yield)#接收send发送的数据
        print('收到任务: ', data)


def producer():
    c = consumer()
    print(c.__next__())  # 返回值为none
    for i in range(3):
        print('发送一个任务...', '任务%d' % i)
        c.send('任务%d' % i)


producer()

等待接受处理任务...
None
发送一个任务... 任务0
收到任务:  任务0
发送一个任务... 任务1
收到任务:  任务1
发送一个任务... 任务2
收到任务:  任务2
